# Análise de livros lidos

Sempre gostei muito de ler e, desde mais ou menos 2013, comecei a anotar minhas leituras no aplicativo Goodreads (antes disso, cheguei a criar várias planilhas de controle da minha biblioteca pessoal, mas infelizmente perdi todos os arquivos).

Como exercício de análise dados, decidi olhar para esse histórico para responder algumas perguntas que, de quebra, podem me ajudar a retomar o hábito de leitura, que já foi mais forte:
- Em que ano li mais, considerando o total de livros lidos?
- Em que ano li mais, considerando o total de _páginas_ lidas?
- Quais foram os anos em que li mais e menos, considerando esses dois critérios acima?
- Nos anos em que li mais, quais foram os gêneros principais?
- Nos anos que li menos, quais foram os gêneros principais?

Bora lá:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('goodreads_library_export.csv')
df.head(1)

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,...,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Owned Copies
0,22896498,Os segredos de Colin Bridgerton (Os Bridgerton...,Julia Quinn,"Quinn, Julia",Claudia Guimarães,"=""8580413079""","=""9788580413076""",0,3.88,Arqueiro,...,NaN,2025/03/19,"curso-artô, artô","curso-artô (#26), artô (#24)",artô,NaN,NaN,NaN,0,0


Dando uma primeira olhada no dataframe, algumas perguntas adicionais me ocorrem, só por curiosidade:

- Quão diferente é o meu gosto do gosto geral, considerando as colunas "My Rating" e "Average Rating"? Isso muda por gênero?
- Quais são minhas editoras preferidas, considerando média de Rating e total de livros lidos?
- Quanto tempo em média eu demoro entre acrescentar um livro à minha lista e de fato lê-lo?

Mas por enquanto vamos focar nas perguntas originais.

## Me familiarizando com os dados

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Book Id                     556 non-null    int64  
 1   Title                       556 non-null    object 
 2   Author                      556 non-null    object 
 3   Author l-f                  556 non-null    object 
 4   Additional Authors          231 non-null    object 
 5   ISBN                        556 non-null    object 
 6   ISBN13                      556 non-null    object 
 7   My Rating                   556 non-null    int64  
 8   Average Rating              556 non-null    float64
 9   Publisher                   541 non-null    object 
 10  Binding                     549 non-null    object 
 11  Number of Pages             546 non-null    float64
 12  Year Published              551 non-null    float64
 13  Original Publication Year   519 non

Colunas com valores nulos com as quais teremos que lidar
- Publisher
- Number of Pages
- Year Published

Podemos tentar buscar um database no Kaggle para complementar essas informações.

As outras colunas com valores nuloss ou têm um motivo claro (ex.: "Date Read" nulo indica que ainda não li), ou não serão usadas na análise.

In [35]:
# Tratando os tipos das colunas de data:
df['Date Read'] = pd.to_datetime(df['Date Read'])
df['Date Added'] = pd.to_datetime(df['Date Added'])

In [38]:
df.describe()

,Book Id,My Rating,Average Rating,Number of Pages,Year Published,Original Publication Year,Date Read,Date Added,Read Count,Owned Copies
count,5.560000e+02,556.000000,556.000000,546.000000,551.000000,519.000000,353,556,556.000000,556.000000
mean,1.980152e+07,1.559353,3.967428,281.935897,2007.735027,1967.527938,2014-12-01 08:17:40.623229440,2017-05-08 16:16:24.172662016,0.861511,0.070144
min,1.100000e+01,0.000000,0.000000,0.000000,1900.000000,-400.000000,2003-01-01 00:00:00,2012-12-09 00:00:00,0.000000,0.000000
25%,8.292480e+05,0.000000,3.810000,180.000000,2005.000000,1979.000000,2011-07-01 00:00:00,2013-02-18 00:00:00,1.000000,0.000000
50%,1.140460e+07,0.000000,4.005000,261.000000,2009.000000,2005.000000,2014-08-10 00:00:00,2015-10-12 00:00:00,1.000000,0.000000
75%,2.634046e+07,3.000000,4.162500,351.500000,2015.000000,2012.000000,2018-06-01 00:00:00,2021-01-24 12:00:00,1.000000,0.000000
max,2.109251e+08,5.000000,5.000000,1234.000000,2024.000000,2023.000000,2025-03-13 00:00:00,2025-03-19 00:00:00,4.000000,1.000000
std,2.873709e+07,1.854647,0.397326,154.683262,13.880615,176.576918,NaN,NaN,0.523908,0.255619


In [5]:
df.shape

(556, 24)

São 556 linhas, mas será que existe algum livro duplicado? Vamos verificar pelo ISBN, um identificador único:

In [24]:
df['ISBN13'].value_counts()

ISBN13
=""                 81
="9788580413076"     1
="9780061125270"     1
="9788508070992"     1
="9788503001373"     1
                    ..
="9788551001288"     1
="9788595085930"     1
="9788571647879"     1
="9788537805558"     1
="9781580495806"     1
Name: count, Length: 476, dtype: int64

Existem livros cadastrados no Goodreads sem ISBN, o que eu não sabia que era possível. Vamos verificar quais são eles para ver se há algum erro:

In [32]:
df[df['ISBN13'] == "=\"\"" ]

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,...,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Owned Copies
3,197377650,Fool Proof Romance: Love Between the Beat Sheets,Christopher Downing,"Downing, Christopher",NaN,"=""""","=""""",0,4.67,Mad Devil Media,...,NaN,2025/03/19,"curso-artô, artô","curso-artô (#23), artô (#21)",artô,NaN,NaN,NaN,0,0
12,63208986,Pequenas Coisas como Estas,Claire Keegan,"Keegan, Claire",Inês Dias,"=""""","=""""",0,4.14,Relógio D’Água,...,NaN,2025/03/19,"curso-artô, to-read","curso-artô (#14), to-read (#115)",to-read,NaN,NaN,NaN,0,0
38,54844350,Boa Sorte,Helena Cunha,"Cunha, Helena",NaN,"=""""","=""""",0,4.38,NaN,...,2024/05/01,2024/06/16,NaN,NaN,read,NaN,NaN,NaN,1,0
44,60883333,Mandíbula,Mónica Ojeda,"Ojeda, Mónica",Silvia Massimini Felix,"=""""","=""""",0,3.69,Autêntica Contemporânea,...,2024/06/02,2024/04/27,NaN,NaN,read,NaN,NaN,NaN,1,0
45,25190667,Os Vingadores (Os Heróis Mais Poderosos da Mar...,Kurt Busiek,"Busiek, Kurt","George Pérez, Stan Lee, Jack Kirby, Eduardo Sa...","=""""","=""""",0,3.72,Salvat,...,2015/11/15,2015/10/16,NaN,NaN,read,NaN,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,5907,"The Hobbit, or There and Back Again",J.R.R. Tolkien,"Tolkien, J.R.R.","Douglas A. Anderson, Michael Hague, Jemima Catlin","=""""","=""""",4,4.29,Houghton Mifflin,...,NaN,2013/02/01,NaN,NaN,read,NaN,NaN,NaN,1,0
532,43035,White Fang,Jack London,"London, Jack",K.A. Applegate,"=""""","=""""",5,4.03,Scholastic Paperbacks,...,NaN,2013/01/30,NaN,NaN,read,NaN,NaN,NaN,4,1
540,12938,King Lear,William Shakespeare,"Shakespeare, William",William James Rolfe,"=""""","=""""",0,3.91,Simon & Schuster,...,2012/01/01,2013/02/01,NaN,NaN,read,NaN,NaN,NaN,1,0
553,1103849,O Homem Duplicado,José Saramago,"Saramago, José",NaN,"=""""","=""""",0,3.97,Editorial Caminho,...,2011/01/01,2013/01/30,NaN,NaN,read,NaN,NaN,NaN,1,0


Realmente temos livros sem ISBN cadastrado, não é um erro. Isso vai complicar um pouco a etapa de remover duplicatas, pois pode ser que livros diferentes tenham o mesmo título. Para verificar, vamos contar os valores únicos na coluna "Title":

In [39]:
df['Title'].nunique()

554

Temos um caso! Vamos procurar qual é ele:

In [42]:
df['Title'].value_counts()

Title
Naked Heat (Nikki Heat, #2)                                                       2
Carry On (Simon Snow, #1)                                                         2
Os segredos de Colin Bridgerton (Os Bridgertons, #4)                              1
O Gênio do Crime                                                                  1
Narrative across Media: The Languages of Storytelling (Frontiers of Narrative)    1
                                                                                 ..
Crisálida                                                                         1
Teocrasília, Livro 1: O Fim da Inocência                                          1
Mitologia Nórdica                                                                 1
Por que não pediram a Evans?                                                      1
The Importance of Being Earnest                                                   1
Name: count, Length: 554, dtype: int64

Achamos dois casos. Meu palpite, sabendo os títulos, é que cadastrei uma versão em inglês e uma versão em português. Vamos verificar:

In [45]:
df[(df['Title'] == 'Carry On (Simon Snow, #1)') | (df['Title'] == 'Naked Heat (Nikki Heat, #2)')]

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,...,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Owned Copies
246,9536649,"Naked Heat (Nikki Heat, #2)",Richard Castle,"Castle, Richard",NaN,"=""078689136X""","=""9780786891368""",3,3.79,Hyperion,...,2019-08-26,2019-08-09,NaN,NaN,read,NaN,NaN,NaN,1,0
289,32768522,"Carry On (Simon Snow, #1)",Rainbow Rowell,"Rowell, Rainbow",NaN,"=""1250135028""","=""9781250135025""",4,4.16,Wednesday Books,...,NaT,2017-07-14,NaN,NaN,read,NaN,NaN,NaN,1,0
293,34705485,"Carry On (Simon Snow, #1)",Rainbow Rowell,"Rowell, Rainbow",NaN,"=""8542809483""","=""9788542809480""",4,4.16,Novo Século,...,2017-07-14,2014-12-10,NaN,NaN,read,NaN,NaN,NaN,1,0
445,7776678,"Naked Heat (Nikki Heat, #2)",Richard Castle,"Castle, Richard",NaN,"=""1401324029""","=""9781401324025""",3,3.79,Hyperion,...,2011-01-01,2013-01-03,NaN,NaN,read,NaN,NaN,NaN,2,0


Meu palpite se confirmou no caso de "Carry On" (pude verificar pelo nome da editora, uma brasileira e uma dos EUA), mas não no caso de "Naked Heat".

Procurando pelos dois ISBNs de Naked Heat no Google, entendi o que aconteceu: um ISBN se refere à edição de capa dura e o outro, à edição paperback. Como sei que só li o paperback, provavelmente reli o livro e acidentalmente registrei na edição errada.

Por isso, farei os seguintes tratamentos:

- Para "Carry On", vou excluir a entrada de índice 289, porque me lembro de só ter lido em português, o que é reforçado pela falta de valor na coluna "Date Read" da versão em inglês.

- Para "Naked Heat", vou manter apenas entrada de índice 246 (paperback) e somar o total de releituras das duas entradas. Comparando o site original do Goodreads e o arquivo csv que ele me entrega, verifiquei que, para livros que foram lidos mais de uma vez, ele registra apenas a primeira data de leitura.[1] Então, vou seguir esse padrão ao tratar os dados.

[1] Isso traz outra implicação ao fazer as análises para responder às perguntas: anos em que eu reli muitos livros terão sua contagem de leitura reduzida nessa análise.

In [56]:
df[df['ISBN13'] == "=\"9780786891368\""]

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,...,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Owned Copies
246,9536649,"Naked Heat (Nikki Heat, #2)",Richard Castle,"Castle, Richard",NaN,"=""078689136X""","=""9780786891368""",3,3.79,Hyperion,...,2019-08-26,2019-08-09,NaN,NaN,read,NaN,NaN,NaN,1,0
